# 猫狗大战

In [5]:
kaggle_cli_installed = True
data_downloaded = True
data_unzipped = True
data_classified_to_dir = False
outliers_removed = True
test_mode = True

if test_mode :
    dataset_path = './test_data/'
else:
    dataset_path = './data/'

In [6]:
import os
import shutil
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## 下载数据集

In [3]:
if not kaggle_cli_installed:
    !pip install kaggle

In [5]:
if not data_downloaded:
#     !cp kaggle.json /home/ubuntu/.kaggle
    !kaggle competitions download -c dogs-vs-cats-redux-kernels-edition -p data

test.zip: Downloaded 271MB of 271MB
train.zip: Downloaded 544MB of 544MB
sample_submission.csv: Downloaded 111KB of 111KB


## 解压数据集

In [6]:
if not data_unzipped:
    import zipfile
#     train_dataset_zip_path = './data/dogs-vs-cats-redux-kernels-edition/train.zip'
#     test_dataset_zip_path = './data/dogs-vs-cats-redux-kernels-edition/test.zip'
    
    train_dataset_zip_path = './data/train.zip'
    test_dataset_zip_path = './data/test.zip'

    def unzip(save_path, data_path, data_name):
        """
        Unzip wrapper with the same interface as _ungzip
        :param save_path: The path of the zip files
        :param data_path: Path to extract to
        :param data_name: Name of dataset
        """
        if not os.path.exists(save_path):
            print('File {} does not exist'.format(data_name))
            return

        print('Extracting data {} ...'.format(data_name))
        with zipfile.ZipFile(save_path) as zf:
            zf.extractall(data_path)
            print('Extract {} Finish!'.format(data_name))

#         print('Removing {}'.format(data_name))
#         os.remove(save_path)

    if not os.path.exists(dataset_path + 'train'):
        unzip(train_dataset_zip_path, dataset_path, 'train.zip')
    else:
        print('Found {} Data'.format('train.zip'))

    if not os.path.exists(dataset_path + 'test'):
        unzip(test_dataset_zip_path, dataset_path, 'test.zip')
    else:
        print('Found {} Data'.format('test.zip'))


Extracting data train.zip ...
Extract train.zip Finish!
Extracting data test.zip ...
Extract test.zip Finish!


## 预处理数据集

In [1]:
# 展示图片的函数
def align_figures():
    import matplotlib
    from matplotlib._pylab_helpers import Gcf
    from IPython.display import display_html
    import base64
    from ipykernel.pylab.backend_inline import show

    images = []
    for figure_manager in Gcf.get_all_fig_managers():
        fig = figure_manager.canvas.figure
        png = get_ipython().display_formatter.format(fig)[0]['image/png']
        src = base64.encodebytes(png).decode()
        images.append('<img style="margin:0" align="left" src="data:image/png;base64,{}"/>'.format(src))

    html = "<div>{}</div>".format("".join(images))
    show._draw_called = False
    matplotlib.pyplot.close('all')
    display_html(html, raw=True)
    
def show_images(img_list, img_path, col_num=5):
    plt.figure(dpi=150)
    for i in range(len(img_list)):
        img = plt.imread(img_path+img_list[i])
        plt.subplot(len(img_list)//col_num+1, col_num, i+1)
        plt.title(img_list[i], fontsize=8)
        plt.axis('off')
        plt.imshow(img)
    align_figures()

#### 读取正确的异常值

In [7]:
with open('real_outliers.txt','r') as f:
    real_outliers = f.read()
    real_outliers = real_outliers.split('\n')
show_num = 10
print(len(real_outliers))
print(real_outliers)

56
['dog.4367.jpg', 'dog.6475.jpg', 'cat.7564.jpg', 'cat.9171.jpg', 'cat.3658.jpg', 'cat.8921.jpg', 'cat.10700.jpg', 'dog.4127.jpg', 'dog.2578.jpg', 'cat.10636.jpg', 'cat.12272.jpg', 'dog.12155.jpg', 'cat.2457.jpg', 'cat.10365.jpg', 'cat.3868.jpg', 'cat.6822.jpg', 'cat.8087.jpg', 'cat.7377.jpg', 'dog.10747.jpg', 'cat.2337.jpg', 'dog.5604.jpg', 'dog.6725.jpg', 'dog.8898.jpg', 'cat.10712.jpg', 'dog.7076.jpg', 'cat.5351.jpg', 'cat.10029.jpg', 'cat.7372.jpg', 'dog.12376.jpg', 'cat.3672.jpg', 'cat.12476.jpg', 'dog.9517.jpg', 'cat.4308.jpg', 'dog.1043.jpg', 'dog.3889.jpg', 'cat.11184.jpg', 'dog.12148.jpg', 'dog.1308.jpg', 'dog.11266.jpg', 'dog.1773.jpg', 'dog.10801.jpg', 'cat.5418.jpg', 'dog.11299.jpg', 'cat.8456.jpg', 'cat.4338.jpg', 'dog.4507.jpg', 'dog.9418.jpg', 'dog.10190.jpg', 'cat.4688.jpg', 'dog.2614.jpg', 'dog.8736.jpg', 'cat.3216.jpg', 'cat.2520.jpg', 'cat.11607.jpg', 'dog.1194.jpg', 'dog.10161.jpg']


In [6]:
# 查看确认正确性
for i in range(len(real_outliers)//show_num+1):
    show_images(real_outliers[i*show_num:(i+1)*show_num], dataset_path+'train/',5)

### 创建猫狗分类文件夹

In [5]:
if not data_classified_to_dir:
    #  创建train2/dog及train2/cat文件夹
    if not os.path.exists(dataset_path + 'train2/dog'):
        os.makedirs(dataset_path+'train2/dog')
        print('Directory "train2/dog" created!')
    else:
        print('Directory "train2/dog" exists!')
        
    if not os.path.exists(dataset_path + 'val2/dog'):
        os.makedirs(dataset_path+'val2/dog')
        print('Directory "val2/dog" created!')
    else:
        print('Directory "val2/dog" exists!')

    if not os.path.exists(dataset_path + 'train2/cat'):
        os.makedirs(dataset_path+'train2/cat')
        print('Directory "train2/cat" created!')
    else:
        print('Directory "train2/cat" exists!')
        
    if not os.path.exists(dataset_path + 'val2/cat'):
        os.makedirs(dataset_path+'val2/cat')
        print('Directory "val2/cat" created!')
    else:
        print('Directory "val2/cat" exists!')

    # 创建test2/test文件夹
    # if not os.path.exists(dataset_path + 'test2/test'):
    #     os.makedirs(dataset_path+'test2/test')
    #     print('Directory "test2/test" created!')
    # else:
    #     print('Directory "test2/test" exists!')


Directory "train2/dog" created!
Directory "val2/dog" created!
Directory "train2/cat" created!
Directory "val2/cat" created!


### 对文件进行分类，并排除异常值

In [7]:
if not data_classified_to_dir:
    from sklearn.model_selection import train_test_split
    from shutil import copyfile
   
    val_fraction = 0.2
    
    # 对训练文件名进行分类
    train_filenames = os.listdir(dataset_path + 'train')
    cat_filenames = filter(lambda x: x[:3] == 'cat', train_filenames)
    dog_filenames = filter(lambda x: x[:3] == 'dog', train_filenames)
    
    # 删除异常值
    cat_filenames = np.array([filename for filename in cat_filenames if filename not in real_outliers])
    dog_filenames = np.array([filename for filename in dog_filenames if filename not in real_outliers])
    
    # 分割训练集与验证集
    train_cat, val_cat = train_test_split(cat_filenames, test_size=val_fraction)
    train_dog, val_dog = train_test_split(dog_filenames, test_size=val_fraction)
    
    if len(os.listdir(dataset_path + 'train2/cat/')) <=0  :
        for filename in train_cat:
            if os.path.isfile(dataset_path+'train2/cat/'+filename):
                os.unlink(dataset_path+'train2/cat/'+filename)
            copyfile(dataset_path+'train/'+filename, dataset_path+'train2/cat/'+filename)
        print('train cat copyfile success')
    else:
        print('train cat files exists')
    
    
    if len(os.listdir(dataset_path + 'val2/cat/')) <=0 :
        for filename in val_cat:
            if os.path.isfile(dataset_path+'val2/cat/'+filename):
                os.unlink(dataset_path+'val2/cat/'+filename)
            copyfile(dataset_path+'train/'+filename, dataset_path+'val2/cat/'+filename)
        
        print('val cat copyfile success')
    else:
        print('val cat files exists')
        
    
    if len(os.listdir(dataset_path + 'train2/dog/')) <=0 :
        for filename in train_dog:
            if os.path.isfile(dataset_path+'train2/dog/'+filename):
                os.unlink(dataset_path+'train2/dog/'+filename)
            copyfile(dataset_path+'train/'+filename, dataset_path+'train2/dog/'+filename)
        print('train dog copyfile success')
    else:
        print('train dog files exists')
        
        
    if len(os.listdir(dataset_path + 'val2/dog/')) <=0 : 
        for filename in val_dog:
            if os.path.isfile(dataset_path+'val2/dog/'+filename):
                os.unlink(dataset_path+'val2/dog/'+filename)
            copyfile(dataset_path+'train/'+filename, dataset_path+'val2/dog/'+filename)
                
        print('val dog copyfile success')
    else:
        print('val dog files exists')
    # 对测试文件进行关联
    # os.symlink(dataset_path+'test', dataset_path+'test2/test')


train cat files exists
val cat files exists
train dog files exists
val dog files exists


In [8]:
print('Number of train cats :',len(train_cat))
print('Number of validation cats :',len(val_cat))
print('Number of train dogs :',len(train_dog))
print('Number of validation dogs :',len(val_dog))


Number of train cats : 32
Number of validation cats : 9
Number of train dogs : 32
Number of validation dogs : 9


In [8]:
if not data_classified_to_dir:
    from shutil import copyfile
    
     #  创建train2/dog及train2/cat文件夹
    if not os.path.exists(dataset_path + 'train3/cat'):
        os.makedirs(dataset_path+'train3/cat')
        print('Directory "train3/cat" created!')
    else:
        print('Directory "train3/cat" exists!')
        
        
    if not os.path.exists(dataset_path + 'train3/dog'):
        os.makedirs(dataset_path+'train3/dog')
        print('Directory "train3/dog" created!')
    else:
        print('Directory "train3/dog" exists!')
    
    train_filenames = os.listdir(dataset_path + 'train')
    cat_filenames = filter(lambda x: x[:3] == 'cat', train_filenames)
    dog_filenames = filter(lambda x: x[:3] == 'dog', train_filenames)
    
    # 删除异常值
    cat_filenames = np.array([filename for filename in cat_filenames if filename not in real_outliers])
    dog_filenames = np.array([filename for filename in dog_filenames if filename not in real_outliers])
    
    print('Number of Cat files:', len(cat_filenames))
    print('Number of Dog files:', len(dog_filenames))
    
    if len(os.listdir(dataset_path + 'train3/cat/')) <=0 :
        for filename in cat_filenames:
            if os.path.isfile(dataset_path+'train3/cat/'+filename):
                os.unlink(dataset_path+'train3/cat/'+filename)
#             os.symlink(dataset_path+'train/'+filename, dataset_path+'train3/cat/'+filename)
            copyfile(dataset_path+'train/'+filename, dataset_path+'train3/cat/'+filename)
        print('train cat copyfile success')
    else:
        print('train cat files exists')
        
    if len(os.listdir(dataset_path + 'train3/dog/')) <=0:
        for filename in dog_filenames:
            if os.path.isfile(dataset_path+'train3/dog/'+filename):
                os.unlink(dataset_path+'train3/dog/'+filename)
#             os.symlink(dataset_path+'train/'+filename, dataset_path+'train3/dog/'+filename)
            copyfile(dataset_path+'train/'+filename, dataset_path+'train3/dog/'+filename)
        print('train dog copyfile success')
    else:
        print('train dog files exists')
        
    print('分类结束!')

Directory "train3/cat" created!
Directory "train3/dog" created!
Number of Cat files: 12471
Number of Dog files: 12473
train cat copyfile success
train dog copyfile success


In [41]:
# 显示目录结构
#省略文件名
!tree  ./data/train2 
# !tree  ./data/train2 /f 

卷 资料 的文件夹 PATH 列表
卷序列号为 0007-DAC2
E:\PYTHON\UDACITY\CAPSTONE\DOG_VS_CAT\DATA\TRAIN2
├─cat
└─dog


## 单模型迁移学习

In [20]:
# common modules
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
# from keras import backend as K

batch_size = 16
epochs=1


def save_model(model, model_name):
    json_model = model.to_json()
    with open('./models/'+model_name+'.json','w') as f:
        f.write(json_model)
    model.save_weights('./models/'+model_name+'.h5')
    print('Model ', model_name, ' saved!')
    
def predict_test(filename='pred.csv',target_size=(299, 299), batch_size=16, model=None):
    if model is None:
        print('No model passed in!')
        return 
    
    import numpy as np
    import pandas as pd
    from keras.preprocessing.image import ImageDataGenerator
    from tqdm import tqdm
    
    df = pd.read_csv("./data/sample_submission.csv")

    test_gen = ImageDataGenerator()
    test_generator = test_gen.flow_from_directory("./data/test", target_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)
    y_pred = model.predict_generator(test_generator, verbose=1,  steps=test_generator.samples//batch_size)
    np.clip(y_pred ,0.005, 0.995)
    
    for i, fname in enumerate(test_generator.filenames):
        index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
        df.set_value(index-1, 'label', y_pred[i])
    
    df.to_csv(filename, index=None)
    df.head(10)
    print('predict finished')


In [15]:
# use ImageDataGenerator validation_split parameter
gen = image.ImageDataGenerator(validation_split=0.2)

train_gen = gen.flow_from_directory(dataset_path+'train3/', target_size=(299,299), shuffle=True, batch_size=64, subset='training', class_mode='binary')
val_gen = gen.flow_from_directory(dataset_path+'train3/', target_size=(299,299), shuffle=True, batch_size=64, subset='validation', class_mode='binary')


Found 19956 images belonging to 2 classes.
Found 4988 images belonging to 2 classes.


### Xception迁移学习

In [9]:
from keras.applications.xception import Xception

In [10]:
# don't use ImageDataGenerator validation_split parameter
gen = image.ImageDataGenerator()

train_gen = gen.flow_from_directory(dataset_path+'train2/', target_size=(299,299), shuffle=True, batch_size=batch_size, class_mode='binary')
val_gen = gen.flow_from_directory(dataset_path+'val2/', target_size=(299,299), shuffle=True, batch_size=batch_size, class_mode='binary')

Found 64 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


In [12]:
xception_base_model = Xception(weights='imagenet', include_top=False)

xception_x = xception_base_model.output
xception_x = GlobalAveragePooling2D()(xception_x)
xception_predictions = Dense(1, activation='sigmoid')(xception_x)

xception_model = Model(inputs=xception_base_model.inputs, outputs=xception_predictions)


# for layer in xception_base_model.layers:
#     layer.trainable = False


xception_model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])
print('训练全部: ')
xception_model.fit_generator(train_gen, epochs=epochs, verbose=1, validation_data=val_gen, steps_per_epoch=train_gen.samples//batch_size)


训练新增的全链接层: 
Epoch 1/1
4/4 [==============================] - 800s 200s/step - loss: 0.3420 - acc: 0.8906 - val_loss: 0.0260 - val_acc: 1.0000


In [13]:
save_model(xception_model, 'xception_model')

In [22]:
predict_test(filename='pred_xception_model.csv',target_size=(299, 299), batch_size=16, model=xception_model)

Found 12500 images belonging to 1 classes.
  2/781 [..............................] - ETA: 9:33:0 - ETA: 9:28:09

KeyboardInterrupt: 

In [ ]:
for i, layer in enumerate(xception_base_model.layers):
    print(i, layer.name)
    
for layer in xception_model.layers[:249]:
    layer.trainable = False

for layer in xception_model.layers[249:]:
    layer.trainable = True
    
xception_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print('finetune: ')
xception_model.fit_generator(train_gen, epochs=10, verbose=1, validation_data=val_gen, steps_per_epoch=25000//16)


### InceptionV3迁移学习

In [6]:
from keras.applications.inception_v3 import InceptionV3

In [5]:
# don't use ImageDataGenerator validation_split parameter
gen = image.ImageDataGenerator()

train_gen = gen.flow_from_directory(dataset_path+'train2/', target_size=(299,299), shuffle=True, batch_size=batch_size, class_mode='binary')
val_gen = gen.flow_from_directory(dataset_path+'val2/', target_size=(299,299), shuffle=True, batch_size=batch_size, class_mode='binary')

Found 64 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


In [7]:
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)

predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.inputs, outputs=predictions)


# for layer in base_model.layers:
#     layer.trainable = False


model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])
print('训练新增的全链接层: ')
model.fit_generator(train_gen, epochs=epochs, verbose=1, validation_data=val_gen, steps_per_epoch=train_gen.samples//batch_size)



训练新增的全链接层: 
Epoch 1/1
4/4 [==============================] - 315s 79s/step - loss: 0.5354 - acc: 0.6562 - val_loss: 4.1651 - val_acc: 0.6111


In [ ]:
json_model = model.to_json()
with open('./models/InceptionV3_model.json','w') as f:
    f.write(json_model)
    
model.save_weights('./models/InceptionV3_model.h5')


In [2]:
def predict_test(filename='pred.csv',target_size=(299, 299), batch_size=16, model=None):
    if model is None:
        print('No model passed in!')
        return 
    
    import pandas as pd
    from keras.preprocessing.image import ImageDataGenerator
#     from keras.applications.inception_v3 import preprocess_input
    from tqdm import tqdm
    df = pd.read_csv("./data/sample_submission.csv")

    test_gen = ImageDataGenerator()
    test_generator = test_gen.flow_from_directory("./data/test", target_size, shuffle=False, 
                                             batch_size=batch_size, class_mode='binary')
    y_pred = model.predict_generator(test_generator, verbose=1,  steps=test_generator.samples//batch_size)
    y_pred = y_pred.clip(min=0.005, max=0.995)
    
    for i, fname in enumerate(test_generator.filenames):
        index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
        df.set_value(index-1, 'label', y_pred[i])
    
    df.to_csv(filename, index=None)
    df.head(10)
    print('predict finished')
    
#     test_images = os.listdir('./data/test')

#     for i in tqdm(range(len(test_images))):

#         img_path = './data/test/{}.jpg'.format(i+1)
#         img = image.load_img(img_path, target_size=target_size)
#         x = image.img_to_array(img)
#         x = np.expand_dims(x, axis=0)
#         x = preprocess_input(x)

#         preds = model.predict(x)
#         preds = preds.clip(min=0.005, max=0.995)
#     #     index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
#         df.set_value(i, 'label', preds)
#     #     if i % 1000 == 0:
#     #         print('{} predicted'.format(i+1))


#     df.to_csv(dataset_path + 'pred.csv', index=None)
#     df.head(10)
#     print('predict finished')

In [4]:
predict_test(filename='pred_test.csv',target_size=(299, 299), batch_size=16)

No model passed in!


### DenseNet121迁移学习